# 协程
- 参考资料

# 迭代器
- 可迭代(Iterable):直接作用于for循环的变量
- 迭代器(Iterable):不但可以作用于for循环, 还可以被next调用.
- list是典型的可迭代对象, 但不是迭代器
- 通过isinstance进行判断
- 通过iter()进行转换

In [25]:
# 可迭代
l = [i for i in range(10)]
print(l)

# range是个迭代器
for i in range(5):
    print(i)同时grouper实例在yield f

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
1
2
3
4


In [26]:
# 判断是否可迭代

from collections import Iterable
from collections import Iterator

l1 = [1,2,3,4,5]

print(isinstance(l1, Iterable))
print(isinstance(l1, Iterator))

True
False


In [58]:
s = 'asdfkha;gds jjk'

print(isinstance(s, Iterable))
print(isinstance(s, Iterator))

s_iter = iter(s)

print(isinstance(s_iter, Iterable))
print(isinstance(s_iter, Iterator))

# 那么迭代器的话...就能这样了吧↓
while True:
    print(next(s_iter), end='.')

True
False
True
True
a.s.d.f.k.h.a.;.g.d.s. .j.j.k.

StopIteration: 

# 生成器(generator)
- 一边循环, 一边计算下一个元素的机制/算法
- 需要满足三个条件:
    - 每次调用都生成for循环需要的下一个元素
    - 如果打到最后一个, 爆出StopIteration异常
    - 可以被next()函数调用
- 如何制作一个生成器
    - 直接使用
    - 如果函数中包含yield, 则这个函数就叫生成器
    - next调用函数, 遇到yield会返回

In [28]:
# 直接使用生成器

L = [x*x for x in range(5)]  # 放在中括号就是列表生成器

g = (x*x for x in range(5))  # 放在小括号就是生成器

print(type(L))
print(type(g))


<class 'list'>
<class 'generator'>


In [49]:
# 函数案例

def odd():
    print('Step 1')
    print('Step 2')
    print('Step 3')
    return None

a = odd()

Step 1
Step 2
Step 3


In [52]:
# 生成器案例
# 在odd函数中, yield负责返回
def odd():
    print('Step 1')
    yield 1
    print('Step 2')
    yield 2
    print('Step 3')
    yield 3

# odd()是调用生成器, 这里普通函数会打印'Step 1',然而此处并没有, 因为里面有yield, 是生成器.
g = odd()
print("已经调用了")

one = next(g) # 在调用next之前, 生成器一动不动像王八, 调用之后, 会一直执行到yield, 并按本句代码的要求把yield后面的值赋给one.
print(one)

two = next(g)
print(two)

three = next(g)


已经调用了
Step 1
1
Step 2
2
Step 3


In [23]:
# for 循环调用生成器

def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        print(b)
        a, b = b, a+b
        n += 1
        
    return None

fib(5)

1
1
2
3
5


In [24]:
# feibonaqi数列生成器写法

def fib(max):
    n, a, b = 0, 0, 1
    while n < max:
        yield b
        a, b = b, a+b
        n += 1
        
    return None

g = fib(5)  # 记住了, 这里函数一动没动.

for i in range(6):
    rst = next(g)  # 看见next, 生成器被激活, 执行到yield后暂停, 把b的值赋给g, 注意, 是"暂停"!!! 
                   # 直到next再次出现, 再继续奔向下一个yield
    print(rst)

1
1
2
3
5


StopIteration: 

In [53]:
ge = fib(10)
"""
生成器的典型用法就是在for循环里使用
比较常用的典型生成器是range.
"""
# 我认为这里g从一开始仍然是一动不动的, 直到for循环的出现, 它起到了next()的作用
for i in ge:
    print(i, end=',')


1,1,2,3,5,8,13,21,34,55,

# 协程

- 历史历程
    - 3.4引入协程, 用yield实现
    - 3.5引入协程语法
    - 实现协程比较好的包有: 
        - asyncio
        - tornado
        - gevent

- 定义: 协程, 十位非抢占式多任务产生子程序的计算机程序组件, 协程允许不同入口点在不同位置暂停或开始执行程续.

- 从技术角度讲, 协程就是一个你可以暂停执行的函数, 或者干脆把协程理解成生成器.

- 协程的实现:
    - yield调用
    - send调用
    
- 协程的四个状态
    - inspect.getgeneratorstate(...)函数确定, 该函数会返回下列字符串中的一个:
    - GEN_CREATED: 等待开始
    - GEN_RUNNIG: 解释器正在执行
    - GEN_SUSPENDED: 在yield处暂停
    - GEN_CLOSED: 执行结束
    - next预激(prime)

- 携程终止
    - StopIteration
    - 协程中未处理的异常会向上冒泡, 传给next函数或send方法的调用方 (即触发携程对象)
    - 停止协程的一种方式: 发送某个哨符值, 让协程退出. 内置的None和Ellipsis 等常量经常用作哨符值==

- yield from
    - 调用协程为了得到返回值, 协程必须正常终止.
    - 生成器正常终止会发出StopIteration异常.
    - yield from 从内部捕获StopIteration异常
    - 案例v03
    - 委派生成器
        - 包含yield from 表达式的生成器函数
        - 委派生成器在yield from表达式处暂停, 调用方可以直接把数据发给子生成器
        - 子生成器再把产生的值发给调用方
        - 子生成器在最后, 解释器会抛出StopIteration异常, 并且把返回值附加到异常对象上.
        - 案例04

In [6]:
# from functools import wraps

def coroutinue(func):
    '''
    装饰器： 向前执行到第一个`yield`表达式，预激`func`
    :param func: func name
    :return: primer
    '''    
#     @wraps(func)
    def primer(*args, **kwargs):
        # 把装饰器生成器函数替换成这里的primer函数；调用primer函数时，返回预激后的生成器。
        gen = func(*args, **kwargs)        # 调用被被装饰函数，获取生成器对象
        next(gen)  # 预激生成器
        return gen  # 返回生成器, 它正常情况是simple_coro的返回值, 但是由于有yield存在, 所以primer的返回值是一个生成器
    return primer

# 使用方法如下

@coroutinue
def simple_coro(a):
    a = yield
    return a + 2同时grouper实例在yield f
    
    
sc = simple_coro(12)  # 已经预激 --> sc = primer(12), 后者返回经过预激的simple_coro(12)
sc.send(12)


StopIteration: 14

In [9]:
import functools
help(functools.wraps)

Help on function wraps in module functools:

wraps(wrapped, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
    Decorator factory to apply update_wrapper() to a wrapper function
    
    Returns a decorator that invokes update_wrapper() with the decorated
    function as the wrapper argument and the arguments to wraps() as the
    remaining arguments. Default arguments are as for update_wrapper().
    This is a convenience function to simplify applying partial() to
    update_wrapper().



In [34]:
# 协程代码案例1

def simple_coroutine():
    print('Started')
    x = yield
    print('Recive', x)
    
sc = simple_coroutine()

print(11111)
next(sc) # 预激

print(22222)
sc.send('啥玩意?')


11111
Started
22222
Recive 啥玩意?


StopIteration: 

In [36]:
# v03

def gen():
    for c in "AB":
        yield c
# list直接用生成器作为参数
print(list(gen()))

def gen_new():
    yield from "AB"

print(list(gen_new()))协程是指一个过程，这个过程与调用方协作，

['A', 'B']
['A', 'B']


In [41]:
# 案例04, 委派生成器
from collections import namedtuple

'''
解释：
1. 外层 for 循环每次迭代会新建一个 grouper 实例，赋值给 coroutine 变量； grouper 是委派生成器。
2. 调用 next(coroutine)，预激委派生成器 grouper，此时进入 while True 循环，调用子生成器 averager 后，在 yield from 表达式处暂停。
3. 内层 for 循环调用 coroutine.send(value)，直接把值传给子生成器 averager。同时，当前的 grouper 实例（coroutine）在 yield from 表达式处暂停。
4. 内层循环结束后， grouper 实例依旧在 yield from 表达式处暂停，因此， grouper函数定义体中为 results[key] 赋值的语句还没有执行。
5. coroutine.send(None) 终止 averager 子生成器，子生成器抛出 StopIteration 异常并将返回的数据包含在异常对象的value中，yield from 可以直接抓取 StopItration 异常并将异常对象的 value 赋值给 results[key]
'''

ResClass = namedtuple('Res', 'count, average')

# 子生成器
def averager():
    total = 0.0
    count = 0
    average = None

    while True:
        term = yield
        # None是哨兵值
        if term is None:
            break
        total += term
        count += 1
        average = total / count

    return ResClass(count, average)


# 委派生成器
def grouper(storages, key):
    # 这个while 是为了本委派生成器在不断地send时能够重复调用yield from averager(), 没有的话, 第二次send()就无路可走了.
    # 因为第一次send之后, grouper是在yield from 处暂停的, 等待继续往下走, 和子生成器是一个道理.(挺奇怪的,我居然卡在这里大半天) 
    # 说白了, 还是我之前对 while 语句的理解不够透彻, 应该在想象中把它展开来, 看做对内部代码的无限复制粘贴就好了.
    while True:  
        # 获取averager()返回的值
        storages[key] = yield from averager()


# 客户端代码
def client():
    process_data = {
        'boys_2': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys_1': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
    }

    storages = {}
    for k, v in process_data.items():
        # 获得协程
        coroutine = g协程是指一个过程，这个过程与调用方协作，rouper(storages, k)

        # 预激协程
        next(coroutine)

        # 发送数据到协程
        for dt in v:
            coroutine.send(dt)

        # 终止协程
        coroutine.send(None)
    print(storages)

# run
client()

{'boys_2': Res(count=9, average=40.422222222222224), 'boys_1': Res(count=9, average=1.3888888888888888)}


# 还剩下的内容
- xml, json
- re, xpath
- 网络编程： socket, ftp, mail
- http协议, ==http web server 小项目
- django, 详细
- 爬虫, API

# asyncio
- python3.4开始引入标准库当中, 内置对异步IO的支持
- asyncio本身是一个消息循环
- 步骤:
    - 创建消息循环
    - 把携程导入
    - 关闭

In [1]:
import threading
# 引入异步IO包
import asyncio

# 使用协程
@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    print('Start....... (%s)' % threading.currentThread())
    yield from asyncio.sleep(3)
    print('Done........ (%s)' % threading.currentThread())
    print('Hello again! (%s)' % threading.currentThread())

# 启动消息循环    
loop = asyncio.get_event_loop()
# 定义任务
tasks = [hello(), hello()]
# asyncio使用wait等待task执行完毕
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

Hello world! (<_MainThread(MainThread, started 139795397941056)>)
Start....... (<_MainThread(MainThread, started 139795397941056)>)
Hello world! (<_MainThread(MainThread, started 139795397941056)>)
Start....... (<_MainThread(MainThread, started 139795397941056)>)
Done........ (<_MainThread(MainThread, started 139795397941056)>)
Hello again! (<_MainThread(MainThread, started 139795397941056)>)
Done........ (<_MainThread(MainThread, started 139795397941056)>)
Hello again! (<_MainThread(MainThread, started 139795397941056)>)


# async and await
- 为了更好的表示异步IO
- 让协程代码更简洁
- 使用上，可以进行简单的替换
    - 用async替换@asyncio.coroutine
    - await 替换 yield from
    

# aiohttp
- asyncio实现单线程的并发IO，在客户端用处不大
- 在服务器端可以asyncio+corotine配合， 因为http是IO操作
- asyncio实现了tcp, udp, ssl等协议
- aiohttp是给予asyncio实现的http框架
- pip install aiohttp

In [ ]:
# aiohttp案例

# concurrent.futures
- python3新增的库
- 类似其他语言的线程池的概念
- 利用multiprocession实现真正的并行计算
- 核心原理：以子进程的形式， 并行运算多个python解释器
- concurrent.futures.Executor
    - ThreadPoolExecutor
    - ProcessPoolExecutor
    - 执行的时候需要字形选择
- submit(fn, args, kwargs)
    - fn 异步执行的参数
    - args, kwargs
    
## Future
    - 

## current map
- map(fn, iterable, timeout=None)
    - 函数需要异步执行